# 1. Ridership

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp namma_metro.ridership

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

import os
import time
import json
import requests

from fastcore.all import Path
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

import inflection
from traffic_data_bengaluru.utils import *

In [ ]:
#| export

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
#| hide
#| eval: false

data_directory = get_data_directory()

In [ ]:
# | export
def fetch_ridership(sleep_duration: float = 0.1):
    """Fetch ridership data from the BMRC API."""
    time.sleep(sleep_duration)

    token = os.environ.get("BMRC_TOKEN")

    
    url = "https://english.bmrc.co.in:8282/api/users/getridership"
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(url, headers=headers, verify=False)
    response.raise_for_status()
    return response.json()

In [ ]:
#| hide
#| eval: false

ridership = fetch_ridership()
print(json.dumps(ridership, indent=4))

{
    "success": "success",
    "results": [
        {
            "TotalPassengers": 1033577,
            "TotalSmartCards": 495308,
            "StoredValueCard": 494122,
            "OneDayPass": 268,
            "ThreeDayPass": 30,
            "FiveDayPass": 888,
            "Tokens": 259305,
            "TotalQR": 102924,
            "QRNammaMetro": 70109,
            "QRWhatsApp": 34780,
            "QRPaytm": 5545,
            "TotalNCMC": 65364,
            "GroupTicket": 242,
            "RidershipDate": "13-10-2025"
        }
    ]
}


In [ ]:
# | export
def task_fetch_ridership(data_directory: Path):
    logger.info("Fetching ridership ...")

    filepath = data_directory / "namma_metro" / 'raw' / 'ridership.jsonl'
    filepath.parent.mkdir(parents=True, exist_ok=True)

    ridership = fetch_ridership()
    append_to_file(filepath, ridership)

    logger.info(f"Ridership data saved successfully.")

In [ ]:
#| hide
#| eval: false

task_fetch_ridership(data_directory)

2025-10-15 07:21:29,280 [INFO] Fetching ridership ...
2025-10-15 07:21:30,891 [INFO] Record already exists.
2025-10-15 07:21:30,891 [INFO] Ridership data saved successfully.


In [ ]:
#| export

def get_ridership(data_directory: Path):
    filepath = data_directory / "namma_metro" / "raw" / "ridership.jsonl"
    records = []
    with open(filepath, "r") as f:
        for line in f:
            item = json.loads(line)
            results = item.get("results", [])
            if results:
                records.append(results[0])

    df = pd.DataFrame(records)
    df.columns = [inflection.underscore(col) for col in df.columns]

    df['ridership_date'] = pd.to_datetime(df['ridership_date'], format='%d-%m-%Y').dt.date
    df = df.sort_values(by='ridership_date', ascending=False)
    
    cols = ['ridership_date'] + [col for col in df.columns if col != 'ridership_date']
    df = df[cols]
    return df

In [ ]:
#| hide
#| eval: false

get_ridership(data_directory)

,ridership_date,total_passengers,total_smart_cards,stored_value_card,one_day_pass,three_day_pass,five_day_pass,tokens,total_qr,qr_namma_metro,qr_whats_app,qr_paytm,total_ncmc,group_ticket
1,2025-10-13,1033577,495308,494122,268,30,888,259305,102924,70109,34780,5545,65364,242
0,2025-10-11,933538,297891,297354,330,79,128,311567,143606,86498,44928,7815,40890,343
